In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. VERİ YÜKLEME VE ETİKETLEME (FEATURE ENGINEERING) ---
def load_and_label_data():
    # Dosya İsimleri ve Tanımladığımız "BIOS Ayarları" (Manuel Etiketleme)
    # Bu ayarlar modelin "Input"u olacak.
    files_config = {
        "Fans ve apu down.csv": {
            "Label": "Default",
            "Set_PPT": 142,  # Ryzen 7600X Stock PPT
            "Set_CO": 0,     # Curve Optimizer
            "Set_Boost": 0   # Boost Override
        },
        "-30 CCD PBO 95 +200 mhz.csv": {
            "Label": "PBO_Tuned_Max",
            "Set_PPT": 95,
            "Set_CO": -30,
            "Set_Boost": 200
        },
        "65W-85W_75A_150A-25_Per_CCD.csv": {
            "Label": "Eco_Mode_65W",
            "Set_PPT": 65,
            "Set_CO": -25,
            "Set_Boost": 0
        },
        "95W-115W_80A_140A-20_Per_CCD.csv": {
            "Label": "Performance_95W",
            "Set_PPT": 95,
            "Set_CO": -20,
            "Set_Boost": 0
        }
    }

    df_list = []
    
    print("Veriler Yükleniyor ve Etiketleniyor...")
    
    for fname, config in files_config.items():
        if os.path.exists(fname):
            # HWInfo dosyasını temizleyerek oku
            try:
                temp_df = pd.read_csv(fname, low_memory=False, on_bad_lines='skip')
                temp_df.columns = [c.strip() for c in temp_df.columns] # Boşluk temizliği
                
                # Sadece ML için gerekli sütunları alalım (Gürültüyü azaltmak için)
                # Girdiler (Inputs) ve Çıktılar (Targets)
                keep_cols = [
                    'CPU (Tctl/Tdie) [°C]',                # Hedef 1: Sıcaklık
                    'Çekirdek Etkili Saatler (avg) [MHz]', # Hedef 2: Performans
                    'CPU Tüm Güç Tüketimi [W]',            # Hedef 3: Güç
                    'CPU [RPM]',                           # Girdi: Fan Hızı
                    'CPU SoC Güç Tüketimi (SVI3 TFN) [W]'  # Girdi: SoC Yükü
                ]
                
                # Varsa al, yoksa geç
                available_cols = [c for c in keep_cols if c in temp_df.columns]
                temp_df = temp_df[available_cols].dropna() # Boş verileri at
                
                # Manuel Etiketleri (Features) Ekle
                temp_df['Set_PPT_Limit'] = config['Set_PPT']
                temp_df['Set_CO_Value'] = config['Set_CO']
                temp_df['Set_Boost_Override'] = config['Set_Boost']
                temp_df['Config_Label'] = config['Label'] # Görselleştirme için
                
                df_list.append(temp_df)
                print(f"  -> {fname} yüklendi ({len(temp_df)} satır). Ayarlar: PPT={config['Set_PPT']}W, CO={config['Set_CO']}")
                
            except Exception as e:
                print(f"HATA: {fname} okunamadı. Sebebi: {e}")
        else:
            print(f"UYARI: {fname} bulunamadı.")

    # Tüm veriyi birleştir
    full_df = pd.concat(df_list, ignore_index=True)
    return full_df

# --- 2. DATASET BÖLME (%75 Train - %25 Test) ---
df = load_and_label_data()

# [YENİ] ORTAM SICAKLIĞI SİMÜLASYONU (DATA AUGMENTATION)
if 'Ambient_Temp' not in df.columns:
    df['Ambient_Temp'] = 20  # Varsayılan kış sıcaklığı

print("Yaz mevsimi simülasyon verileri üretiliyor...")
df_summer = df.copy()
df_summer['Ambient_Temp'] = 30  # Yaz sıcaklığı

# CPU Sıcaklık sütun isminiz 'CPU (Tctl/Tdie) [°C]' ise:
temp_col = 'CPU (Tctl/Tdie) [°C]'  

if temp_col in df_summer.columns:
    df_summer[temp_col] = (df_summer[temp_col] + 10).clip(upper=95)
    df = pd.concat([df, df_summer], ignore_index=True)
    print(f"Veri seti genişletildi. Yeni satır sayısı: {len(df)}")
else:
    print(f"HATA: '{temp_col}' sütunu bulunamadı. Lütfen sütun adını kontrol edin.")
# =============================================================================

# Model Girdileri (X): Ayarlar + Çevresel Faktörler (Fan Hızı vb.)
# Fan hızını da "Girdi" olarak alıyoruz, böylece "Fan %50 olursa sıcaklık ne olur?" diye sorabileceğiz.
X = df[['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU [RPM]', 'Ambient_Temp']]

# Model Hedefleri (Y): Tahmin etmek istediklerimiz
# 1. Sıcaklık Modeli için
y_temp = df['CPU (Tctl/Tdie) [°C]']
# 2. Performans Modeli için
y_clock = df['Çekirdek Etkili Saatler (avg) [MHz]']

# Bölme İşlemi (Random State 42 = Bilimsel Tekrarlanabilirlik için sabit tohum)
# Shuffle=True (Veriyi karıştır) önemlidir, çünkü zaman serisi verisinde
# "ilk dosyayı train, son dosyayı test" yaparsak model yanılır. Karıştırarak genel karakteristiği öğretiyoruz.
X_train, X_test, y_temp_train, y_temp_test, y_clock_train, y_clock_test = train_test_split(
    X, y_temp, y_clock, test_size=0.25, random_state=42
)

print("\n--- Veri Seti Hazır ---")
print(f"Toplam Veri Sayısı: {len(df)}")
print(f"Eğitim Seti (%75): {len(X_train)} örnek")
print(f"Test Seti (%25):   {len(X_test)} örnek")
print("\nMakine Öğrenimi Modelleri kurulmaya hazır.")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# --- 3. MODEL EĞİTİMİ (TRAINING) ---
print("Modeller Eğitiliyor... (Bu işlem birkaç saniye sürebilir)")

# A) Termal Model (Sıcaklık Tahmincisi)
model_temp = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_temp.fit(X_train, y_temp_train)
print("✅ Termal Model Eğitildi.")

# B) Performans Modeli (MHz Tahmincisi)
model_clock = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_clock.fit(X_train, y_clock_train)
print("✅ Performans Modeli Eğitildi.")

# --- 4. MODEL TESTİ VE DEĞERLENDİRME (EVALUATION) ---
# Test seti üzerinde tahmin yap
pred_temp = model_temp.predict(X_test)
pred_clock = model_clock.predict(X_test)

# Metrikleri Hesapla
# MAE (Mean Absolute Error): Ortalama kaç derece/MHz sapıyoruz?
mae_temp = mean_absolute_error(y_temp_test, pred_temp)
mae_clock = mean_absolute_error(y_clock_test, pred_clock)

# R2 Score: Modelin veriyi açıklama başarısı (1.00 mükemmeldir)
r2_temp = r2_score(y_temp_test, pred_temp)
r2_clock = r2_score(y_clock_test, pred_clock)

print("\n--- MODEL PERFORMANS RAPORU ---")
print(f"🌡️  Termal Model Hatası (MAE): ±{mae_temp:.2f}°C")
print(f"🎯 Termal Model Başarısı (R²): {r2_temp:.4f}")
print("-" * 30)
print(f"🚀 Performans Model Hatası (MAE): ±{mae_clock:.2f} MHz")
print(f"🎯 Performans Model Başarısı (R²): {r2_clock:.4f}")

# --- 5. GÖRSELLEŞTİRME (Doğruluk ve Özellik Önemi) ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
sns.set_style("darkgrid")

# Grafik 1: Gerçek vs Tahmin (Sıcaklık)
sns.scatterplot(x=y_temp_test, y=pred_temp, ax=axes[0,0], alpha=0.5, color='orange')
axes[0,0].plot([y_temp.min(), y_temp.max()], [y_temp.min(), y_temp.max()], 'r--', lw=2) # İdeal çizgi
axes[0,0].set_title(f'Termal Tahmin Doğruluğu (R²={r2_temp:.3f})')
axes[0,0].set_xlabel('Gerçek Sıcaklık (°C)')
axes[0,0].set_ylabel('Tahmin Edilen Sıcaklık (°C)')

# Grafik 2: Gerçek vs Tahmin (Performans)
sns.scatterplot(x=y_clock_test, y=pred_clock, ax=axes[0,1], alpha=0.5, color='green')
axes[0,1].plot([y_clock.min(), y_clock.max()], [y_clock.min(), y_clock.max()], 'r--', lw=2)
axes[0,1].set_title(f'Performans Tahmin Doğruluğu (R²={r2_clock:.3f})')
axes[0,1].set_xlabel('Gerçek Frekans (MHz)')
axes[0,1].set_ylabel('Tahmin Edilen Frekans (MHz)')

# Grafik 3: Özellik Önemi (Sıcaklığı en çok ne etkiliyor?)
feat_imp_temp = pd.Series(model_temp.feature_importances_, index=X.columns).sort_values(ascending=False)
sns.barplot(x=feat_imp_temp.values, y=feat_imp_temp.index, hue=feat_imp_temp.index, legend=False, ax=axes[1,0], palette='Oranges_r')
axes[1,0].set_title('Sıcaklığı Etkileyen Faktörler')

# Grafik 4: Özellik Önemi (Hızı en çok ne etkiliyor?)
feat_imp_clock = pd.Series(model_clock.feature_importances_, index=X.columns).sort_values(ascending=False)
sns.barplot(x=feat_imp_clock.values, y=feat_imp_clock.index, hue=feat_imp_clock.index, legend=False, ax=axes[1,1], palette='Greens_r')
axes[1,1].set_title('Performansı Etkileyen Faktörler')

plt.tight_layout()
plt.show()

In [ ]:
def run_virtual_stress_test(model_t, model_c, current_worst_core_rank):
    """
    Eğitilmiş ML modellerini kullanarak, denenmemiş BIOS ayarlarını simüle eder.
    Ayrıca Silikon Kalitesi verisine göre çökme riski tahmini yapar.
    """
    
    # --- 1. SENARYOLAR (What-If Analysis) ---
    # Buraya denemek istediğiniz hayali ayarları giriyoruz.
    # Fan Hızı (RPM) olarak 1500 (Max) baz alınmıştır.
    scenarios = [
        # Senaryo 1: Aşırı Tasarruf (Laptop Modu gibi)
        {"Name": "Ultra Eco (45W)", "Set_PPT_Limit": 45, "Set_CO_Value": -30, "Set_Boost_Override": 0, "CPU [RPM]": 1000},
        
        # Senaryo 2: "Sweet Spot" Adayı (Analizlerden tahmin ettiğimiz ideal nokta)
        {"Name": "AI Optimized (85W)", "Set_PPT_Limit": 85, "Set_CO_Value": -25, "Set_Boost_Override": 100, "CPU [RPM]": 1500},
        
        # Senaryo 3: Riskli Performans (Limitleri zorla)
        {"Name": "Extreme OC (130W)", "Set_PPT_Limit": 130, "Set_CO_Value": -30, "Set_Boost_Override": 200, "CPU [RPM]": 2000},
        
        # Senaryo 4: İmkansız Undervolt (Çökme testi)
        {"Name": "Suicide Run (-40 CO)", "Set_PPT_Limit": 105, "Set_CO_Value": -40, "Set_Boost_Override": 150, "CPU [RPM]": 2000},
        
        # Senaryo 5: Sessiz Performans
        {"Name": "Silent Gaming", "Set_PPT_Limit": 95, "Set_CO_Value": -20, "Set_Boost_Override": 50, "CPU [RPM]": 800}
    ]
    
    # DataFrame oluştur
    sim_df = pd.DataFrame(scenarios)
    sim_df['Ambient_Temp'] = 20
    
    # --- 2. YAPAY ZEKA TAHMİNİ (PREDICTION) ---
    # Modelin beklediği özellik sırası: ['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU [RPM]']
    X_sim = sim_df[['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU [RPM]', 'Ambient_Temp']]
    
    print("Yapay Zeka Senaryoları İşliyor...")
    
    # Tahminleri yap
    pred_temps = model_t.predict(X_sim)
    pred_clocks = model_c.predict(X_sim)
    
    # Sonuçları tabloya ekle
    sim_df['Tahmini Sıcaklık (°C)'] = pred_temps
    sim_df['Tahmini Hız (MHz)'] = pred_clocks
    sim_df['Verimlilik (MHz/°C)'] = sim_df['Tahmini Hız (MHz)'] / sim_df['Tahmini Sıcaklık (°C)']
    
    # --- 3. RİSK ANALİZİ (Logic Based on Silicon Lottery) ---
    # Core 5'in kalitesi düşük olduğu için agresif CO değerlerinde uyarı verelim.
    def assess_risk(row):
        risk_score = 0
        reasons = []
        
        # Risk 1: Curve Optimizer Sınırı
        # Bizim analizimizde Core 5 (-20) istiyordu. -25 ve altı risklidir.
        if row['Set_CO_Value'] < -25: 
            risk_score += 5
            reasons.append(f"ÇÖKME RİSKİ: Zayıf Çekirdek (Core 5) -25 altını kaldıramayabilir.")
        elif row['Set_CO_Value'] < -20:
            risk_score += 2
            reasons.append("Uyarı: Core 5 kararsızlaşabilir.")
            
        # Risk 2: Termal Sınır
        if row['Tahmini Sıcaklık (°C)'] > 92:
            risk_score += 3
            reasons.append("Termal Limit: 95°C'ye çok yakın.")
            
        # Risk 3: PPT Yetersizliği
        # Eğer PPT çok düşükse (45W) ve Boost yüksekse, saat hızı düşer (Clock Stretching).
        if row['Set_PPT_Limit'] < 65 and row['Set_Boost_Override'] > 0:
            reasons.append("Verimsiz: Düşük Watt ile Boost yapılamaz.")

        if risk_score >= 5: return "🔴 YÜKSEK RİSK (" + ", ".join(reasons) + ")"
        elif risk_score >= 2: return "🟡 ORTA RİSK (" + ", ".join(reasons) + ")"
        else: return "🟢 GÜVENLİ"

    sim_df['Risk Analizi'] = sim_df.apply(assess_risk, axis=1)
    
    # --- 4. SONUÇ GÖSTERİMİ ---
    display_cols = ['Name', 'Set_PPT_Limit', 'Set_CO_Value', 'Tahmini Sıcaklık (°C)', 'Tahmini Hız (MHz)', 'Risk Analizi']
    
    print("\n--- AI DESTEKLİ SANAL STRES TESTİ SONUÇLARI ---")
    display(sim_df[display_cols].style.background_gradient(cmap='RdYlGn_r', subset=['Tahmini Sıcaklık (°C)'])
                                     .background_gradient(cmap='RdYlGn', subset=['Tahmini Hız (MHz)'])
                                     .format({"Tahmini Sıcaklık (°C)": "{:.1f}", "Tahmini Hız (MHz)": "{:.0f}"}))

# Modelleri önceki hücreden (global) alarak çalıştır
# Worst Core Rank: Bizim analizde Core 5 en kötü çıkmıştı.
run_virtual_stress_test(model_temp, model_clock, current_worst_core_rank=5)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. SÜTUN İSİMLERİNİ TEMİZLEME (XGBoost Hatası Düzeltmesi) ---
# Köşeli parantezleri kaldırıyoruz: "CPU [RPM]" -> "CPU RPM" oluyor.
print("Sütun isimleri XGBoost için temizleniyor...")
df.columns = df.columns.str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.strip()

# İsimler değiştiği için değişkenleri güncelleyelim
# HWInfo Türkçe karakterleri ve birimleri:
col_temp = 'CPU (Tctl/Tdie) °C' 
col_clock = 'Çekirdek Etkili Saatler (avg) MHz'
col_power = 'CPU Tüm Güç Tüketimi W'
col_rpm = 'CPU RPM' # Artık parantez yok

# --- 2. FEATURE ENGINEERING 2.0 (Zaman ve Fizik) ---
def create_advanced_features(input_df):
    df_adv = input_df.copy()
    
    print("Gelişmiş özellikler (Lag & Rolling) oluşturuluyor...")
    
    # Her profili kendi içinde grupla (Veri karışmasını önle)
    grouped = df_adv.groupby('Config_Label')
    
    # A) LAG FEATURES (Geçmiş Bilgisi)
    # 1 satır önceki sıcaklık ve hız
    df_adv['Temp_Lag1'] = grouped[col_temp].shift(1)
    df_adv['Temp_Lag2'] = grouped[col_temp].shift(2)
    df_adv['Clock_Lag1'] = grouped[col_clock].shift(1)
    
    # B) ROLLING FEATURES (Ortalama Trend)
    # Son 5 örneğin güç tüketimi ortalaması
    df_adv['Power_Roll_Mean_5'] = grouped[col_power].transform(lambda x: x.rolling(window=5).mean())
    
    # C) ETKİLEŞİM TERİMLERİ (Fiziksel Oranlar)
    # Soğutma Verimliliği = Sıcaklık / (Fan Hızı + 1)
    df_adv['Fan_Cooling_Efficiency'] = df_adv[col_temp] / (df_adv[col_rpm] + 1)
    
    # Agresiflik Skoru = PPT * CO (Mutlak Değer)
    df_adv['Aggressiveness_Score'] = df_adv['Set_PPT_Limit'] * df_adv['Set_CO_Value'].abs()

    # NaN (Boş) değerleri temizle (Lag işlemi ilk satırları boş bırakır)
    df_adv = df_adv.dropna()
    return df_adv

# Veriyi işle
df_advanced = create_advanced_features(df)

# --- 3. VERİ SETİ BÖLME (%75 Train - %25 Test) ---
# Model Girdileri (X)
feature_cols = [
    'Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', col_rpm, 
    'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 
    'Aggressiveness_Score'
]

X = df_advanced[feature_cols]
y_temp = df_advanced[col_temp]
y_clock = df_advanced[col_clock]

# %75 Eğitim, %25 Test
X_train, X_test, y_temp_train, y_temp_test, y_clock_train, y_clock_test = train_test_split(
    X, y_temp, y_clock, test_size=0.25, random_state=42, shuffle=True
)

# --- 4. XGBOOST MODEL EĞİTİMİ (ORDINARYUS MODU: BAYESIAN OPTIMIZATION) ---
import optuna
from sklearn.model_selection import cross_val_score, KFold

# Log kirliliğini önle
optuna.logging.set_verbosity(optuna.logging.WARNING)

print("\n🎓 ORDINARYUS MODU: Parametreler Otomatik Optimize Ediliyor...")
print("="*60)

def optimize_xgboost(X, y, target_name="Target"):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0.0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
            'n_jobs': -1,
            'random_state': 42
        }
        
        model = xgb.XGBRegressor(**params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error')
        return -scores.mean()

    print(f"🔍 {target_name} için en iyi parametreler aranıyor (Bu işlem biraz sürebilir)...")
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=30) # Hız için 30 deneme yeterli (İstersen 50 yap)
    
    print(f"   ✅ {target_name} En İyi RMSE: {study.best_value:.4f}")
    return study.best_params

# 1. Optimize Et
best_params_temp = optimize_xgboost(X_train, y_temp_train, "Sıcaklık (Temp)")
best_params_clock = optimize_xgboost(X_train, y_clock_train, "Hız (Clock)")

print("\n🚀 FİNAL EĞİTİM: Bulunan Mükemmel Parametrelerle Modeller Kuruluyor...")

# 2. Final Modelleri Eğit
# (Değişken isimlerini eski kodunla uyumlu hale getirdim)
model_temp_xgb = xgb.XGBRegressor(**best_params_temp)
model_temp_xgb.fit(X_train, y_temp_train)

model_clock_xgb = xgb.XGBRegressor(**best_params_clock)
model_clock_xgb.fit(X_train, y_clock_train)

# --- 5. ORDINARYUS MODEL PERFORMANSI ---
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error

# Test seti üzerinde tahmin yap
pred_temp = model_temp_xgb.predict(X_test)
pred_clock = model_clock_xgb.predict(X_test)

# Skorları Hesapla
r2_temp = r2_score(y_temp_test, pred_temp)
r2_clock = r2_score(y_clock_test, pred_clock)
mae_temp = mean_absolute_error(y_temp_test, pred_temp)

print("\n📊 MODEL KARNESİ:")
print(f"   🌡️  Sıcaklık Tahmin Başarısı (R²): %{r2_temp*100:.2f}")
print(f"   🚀 Hız Tahmin Başarısı (R²):     %{r2_clock*100:.2f}")
print(f"   📉 Sıcaklık Hata Payı:          ±{mae_temp:.2f} °C")

# GÖRSELLEŞTİRME
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# 1. Gerçek vs Tahmin (Sıcaklık)
sns.scatterplot(x=y_temp_test, y=pred_temp, ax=ax[0], color='orange', alpha=0.5)
ax[0].plot([y_temp.min(), y_temp.max()], [y_temp.min(), y_temp.max()], 'r--', lw=2)
ax[0].set_title(f"Sıcaklık: Gerçek vs Yapay Zeka (R²={r2_temp:.3f})")
ax[0].set_xlabel("Gerçek Sıcaklık (°C)")
ax[0].set_ylabel("Tahmin Edilen (°C)")

# 2. Hangi Özellik Daha Önemli? (XGBoost Feature Importance)
# Modelin hangi ayara daha çok dikkat ettiğini gösterir
xgb.plot_importance(model_clock_xgb, ax=ax[1], max_num_features=10, height=0.5, 
                    color='purple', title='Hız İçin En Kritik Faktörler')
ax[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
# --- 7. DİNAMİK STRES TESTİ SİMÜLASYONU (ORDINARYUS UYUMLU) ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def simulate_dynamic_stress_test_ordinaryus(model_t, model_c, scenarios, steps=60):
    """
    Ordinaryus modelin 'Lag' (Geçmiş) özelliğini kullanarak
    zaman içinde ısınmayı simüle eder (Feedback Loop).
    """
    simulation_results = []
    
    print(f"🎬 Dinamik Simülasyon Başlıyor ({steps} saniye)...")
    
    for scen in scenarios:
        # 1. BAŞLANGIÇ KOŞULLARI (Cold Start)
        # İşlemci testi başlattığında durumu nedir?
        current_temp = 35.0      # Başlangıç sıcaklığı
        prev_temp = 30.0         # Lag2 için
        current_clock = 3000.0   # Başlangıç hızı (Base clock)
        
        # Power History (Rolling Mean hesabı için son 5 saniyenin gücü)
        # Başlangıçta PPT limitinin yarısı kadar güç çektiğini varsayalım
        power_history = [scen['Set_PPT_Limit'] * 0.5] * 5 
        
        label = f"PPT {scen['Set_PPT_Limit']}W | CO {scen['Set_CO_Value']}"
        
        for t in range(steps):
            # 2. ANLIK GÜÇ HESABI (Basit Fizik)
            # Sıcaklık arttıkça güç tüketimi hafif artar (Leakage current)
            # Ama PPT limitini asla geçemez.
            estimated_power = min(scen['Set_PPT_Limit'], scen['Set_PPT_Limit'] * 0.9 + (current_temp * 0.1))
            
            # Tarihçeyi güncelle ve ortalama al
            power_history.append(estimated_power)
            power_history.pop(0)
            power_roll_mean = sum(power_history) / 5
            
            # 3. MODEL GİRDİLERİNİ HAZIRLA
            # Eğitimdeki 'feature_cols' sırasıyla birebir aynı olmalı!
            input_row = pd.DataFrame([{
                'Set_PPT_Limit': scen['Set_PPT_Limit'],
                'Set_CO_Value': scen['Set_CO_Value'],
                'Set_Boost_Override': scen.get('Set_Boost_Override', 0), # Yoksa 0 al
                'CPU RPM': scen.get('CPU RPM', 1800),                    # Yoksa 1800 al
                
                # --- İŞTE KRİTİK NOKTA (FEEDBACK LOOP) ---
                'Temp_Lag1': current_temp,   # Bir önceki tahmin buraya giriyor
                'Temp_Lag2': prev_temp,      # İki önceki tahmin
                'Clock_Lag1': current_clock, # Bir önceki hız
                
                'Power_Roll_Mean_5': power_roll_mean,
                'Aggressiveness_Score': scen['Set_PPT_Limit'] * abs(scen['Set_CO_Value'])
            }])
            
            # 4. ORDINARYUS TAHMİNİ
            pred_temp = model_t.predict(input_row)[0]
            pred_clock = model_c.predict(input_row)[0]
            
            # Sonuçları kaydet
            simulation_results.append({
                'Time_Sec': t,
                'Label': label,
                'Temp_C': pred_temp,
                'Clock_MHz': pred_clock,
                'PPT_Limit': scen['Set_PPT_Limit']
            })
            
            # 5. DURUMU GÜNCELLE (BİR SONRAKİ ADIM İÇİN)
            prev_temp = current_temp
            current_temp = pred_temp
            current_clock = pred_clock

    return pd.DataFrame(simulation_results)

# --- SENARYOLARI TANIMLA ---
test_scenarios = [
    {'Set_PPT_Limit': 142, 'Set_CO_Value': 0,   'Set_Boost_Override': 0, 'CPU RPM': 1800}, # Stok
    {'Set_PPT_Limit': 88,  'Set_CO_Value': -20, 'Set_Boost_Override': 100, 'CPU RPM': 1800}, # Bizim Ayar
    {'Set_PPT_Limit': 65,  'Set_CO_Value': -30, 'Set_Boost_Override': 0, 'CPU RPM': 1800}  # Eco
]

# --- SİMÜLASYONU ÇALIŞTIR ---
df_dynamic = simulate_dynamic_stress_test_ordinaryus(
    model_temp_xgb, 
    model_clock_xgb, 
    test_scenarios, 
    steps=60
)

# --- GRAFİK ÇİZ ---
plt.figure(figsize=(14, 6))

# Sol: Isınma Eğrisi
plt.subplot(1, 2, 1)
sns.lineplot(data=df_dynamic, x='Time_Sec', y='Temp_C', hue='Label', linewidth=2.5)
plt.axhline(95, color='red', linestyle='--', label='Thermal Limit (95°C)')
plt.title("Zaman İçinde Isınma (0-60 Saniye)", fontsize=12)
plt.ylabel("Sıcaklık (°C)")
plt.xlabel("Zaman (Saniye)")
plt.grid(True, alpha=0.3)

# Sağ: Performans Eğrisi
plt.subplot(1, 2, 2)
sns.lineplot(data=df_dynamic, x='Time_Sec', y='Clock_MHz', hue='Label', linewidth=2.5)
plt.title("Zaman İçinde Performans Kararlılığı", fontsize=12)
plt.ylabel("Hız (MHz)")
plt.xlabel("Zaman (Saniye)")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# --- 8. MEVSİMSEL PROFİL SİMÜLASYONU (ORDINARYUS UYUMLU) ---
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def generate_seasonal_profiles_ordinaryus(model_t, model_c):
    print("🌍 Mevsimsel Senaryolar Hesaplanıyor...")
    
    # 1. Kullanıcı Profilleri (Standart Ayarlar)
    profiles = [
        # Eco: Az yakar, serin çalışır
        {'Label': 'Eco Mode',       'PPT': 65,  'CO': -30, 'Boost': 0,   'RPM': 1500},
        # Balanced: İdeal ev kullanıcısı
        {'Label': 'Balanced',       'PPT': 88,  'CO': -20, 'Boost': 100, 'RPM': 1800},
        # Performance: Sınırları zorlar
        {'Label': 'Performance',    'PPT': 120, 'CO': -20, 'Boost': 200, 'RPM': 2000},
        # Stock: Fabrika ayarı
        {'Label': 'Stock (Ref)',    'PPT': 142, 'CO': 0,   'Boost': 0,   'RPM': 2000}
    ]
    
    # 2. Mevsimler (Ortam Sıcaklığı Farkları)
    # Modelimiz 24°C ortam sıcaklığında eğitildi varsayıyoruz.
    # Farkı manuel ekleyeceğiz (Delta T).
    seasons = [
        {'Season': '❄️ Kış (20°C)',   'Ambient_Delta': -4}, # 24 - 4 = 20
        {'Season': '☀️ Yaz (30°C)',   'Ambient_Delta': +6}, # 24 + 6 = 30
        {'Season': '🔥 Sıcak (35°C)', 'Ambient_Delta': +11} # 24 + 11 = 35
    ]
    
    results = []

    for season in seasons:
        for prof in profiles:
            # --- MODEL GİRDİLERİ (Feature Engineering) ---
            # Steady State (Kararlı Hal) Varsayımı:
            # İşlemci zaten ısınmış, fanlar dönüyor.
            
            # Tahmini Kararlı Sıcaklık (Lag için)
            # Basit fizik: PPT arttıkça sıcaklık artar varsayımıyla Lag dolduruyoruz.
            estimated_steady_temp = 45 + (prof['PPT'] * 0.3)
            
            input_row = pd.DataFrame([{
                'Set_PPT_Limit': prof['PPT'],
                'Set_CO_Value': prof['CO'],
                'Set_Boost_Override': prof['Boost'],
                'CPU RPM': prof['RPM'],
                
                # Ordinaryus Özellikleri
                'Temp_Lag1': estimated_steady_temp,
                'Temp_Lag2': estimated_steady_temp,
                'Clock_Lag1': 4800, # Ortalama bir hız varsayımı
                'Power_Roll_Mean_5': prof['PPT'] * 0.95, # Limite yakın güç çekiyor
                'Aggressiveness_Score': prof['PPT'] * abs(prof['CO'])
            }])
            
            # Tahmin
            base_temp = model_t.predict(input_row)[0]
            pred_clock = model_c.predict(input_row)[0]
            
            # Mevsim Etkisini Ekle (Fiziksel Düzeltme)
            # Ortam sıcaklığı artarsa işlemci sıcaklığı da artar.
            final_temp = base_temp + season['Ambient_Delta']
            
            # Thermal Throttling Kontrolü (95 Dereceyi geçerse hız düşer)
            if final_temp > 95:
                final_temp = 95
                # Her 1 derece aşım için kabaca 50 MHz hız kaybı (Throttling)
                throttling_loss = (base_temp + season['Ambient_Delta'] - 95) * 50
                pred_clock -= throttling_loss
            
            results.append({
                'Profil': prof['Label'],
                'Mevsim': season['Season'],
                'Sıcaklık (°C)': final_temp,
                'Hız (MHz)': pred_clock,
                'Verimlilik': pred_clock / final_temp # Basit skor
            })

    df_results = pd.DataFrame(results)
    
    # --- GÖRSELLEŞTİRME ---
    plt.figure(figsize=(12, 6))
    
    # Bar Chart: Her mevsimde sıcaklıklar nasıl?
    sns.barplot(
        data=df_results, 
        x='Profil', 
        y='Sıcaklık (°C)', 
        hue='Mevsim', 
        palette='coolwarm'
    )
    
    plt.axhline(95, color='red', linestyle='--', label='Throttle Limiti (95°C)')
    plt.axhline(70, color='green', linestyle='--', label='Konfor Alanı (70°C)')
    plt.title("Mevsimlere Göre Ryzen 7600X Profilleri", fontsize=14)
    plt.ylabel("Tahmini Sıcaklık (°C)")
    plt.xlabel("Kullanıcı Profili")
    plt.legend(title='Ortam Koşulu')
    plt.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df_results

# Çalıştır
df_seasons = generate_seasonal_profiles_ordinaryus(model_temp_xgb, model_clock_xgb)

In [ ]:
# --- 9. YÜKSEK FREKANS PROFİLLERİ (ORDINARYUS UYUMLU) ---
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def simulate_high_frequency_profiles_ordinaryus(model_t, model_c):
    print("🚀 Yüksek Frekans (Overclock) Potansiyeli Taranıyor...")
    
    scenarios = []
    
    # Sadece Yüksek Performans Ayarlarını Tara
    # PPT: 90W'dan başlayıp limitleri zorluyoruz (160W'a kadar)
    # CO: Sadece negatif değerler (Voltaj düşürmeden yüksek hız olmaz)
    # Boost: SADECE +200 MHz (Rekor için şart)
    
    for ppt in range(90, 165, 5):
        for co in [-30, -25, -20, -15, -10]:
            scenarios.append({
                'Set_PPT_Limit': ppt,
                'Set_CO_Value': co,
                'Set_Boost_Override': 200, # Kilit nokta
                'CPU RPM': 2000            # Fanlar %100
            })
            
    df_sim = pd.DataFrame(scenarios)
    
    # --- MODEL GİRDİLERİ (Zorlanmış Varsayımlar) ---
    # Modele "İşlemci şu an yarış modunda" diyoruz.
    
    # 1. Fiziksel Özellikler
    df_sim['Aggressiveness_Score'] = df_sim['Set_PPT_Limit'] * df_sim['Set_CO_Value'].abs()
    df_sim['Power_Roll_Mean_5'] = df_sim['Set_PPT_Limit'] # Limitlerde güç çekiyor
    
    # 2. Lag Değerleri (Yarış Modu Varsayımı)
    # İşlemcinin zaten 5000 MHz üstüne çıktığını ve ısındığını varsayıyoruz.
    # Modelden "Bir sonraki adımda 5300 olur mu?" cevabını istiyoruz.
    df_sim['Clock_Lag1'] = 5200.0 
    df_sim['Temp_Lag1'] = 80.0
    df_sim['Temp_Lag2'] = 80.0
    
    # Sütun Sırası (Hata önleyici)
    required_cols = [
        'Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU RPM', 
        'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 
        'Aggressiveness_Score'
    ]
    
    # --- TAHMİN ---
    X_sim = df_sim[required_cols]
    df_sim['Pred_Temp'] = model_t.predict(X_sim)
    df_sim['Pred_Clock'] = model_c.predict(X_sim)
    
    # --- SONUÇ ANALİZİ ---
    # 5.3 GHz (5300 MHz) Barajını geçen var mı?
    elite_tier = df_sim[df_sim['Pred_Clock'] >= 5300]
    
    print(f"\nTarama Sonucu: {len(df_sim)} senaryo içinden {len(elite_tier)} tanesi 5.3 GHz barajını geçti.")
    
    if not elite_tier.empty:
        best_of_best = elite_tier.sort_values(by='Pred_Temp', ascending=True).iloc[0]
        print("\n🏆 ALTIN ÇİP AYARI BULUNDU! (En Serin 5.3+ GHz)")
        print(f"   • Hız:      {best_of_best['Pred_Clock']:.0f} MHz")
        print(f"   • Sıcaklık: {best_of_best['Pred_Temp']:.1f} °C")
        print(f"   • Ayarlar:  PPT {best_of_best['Set_PPT_Limit']}W | CO {best_of_best['Set_CO_Value']} | Boost +200")
    else:
        # Eğer 5300 yoksa en yükseği göster
        best_effort = df_sim.sort_values(by='Pred_Clock', ascending=False).iloc[0]
        print("\n⚠️ 5.3 GHz Barajı Aşılamadı. En Yüksek Potansiyel:")
        print(f"   • Max Hız:  {best_effort['Pred_Clock']:.0f} MHz")
        print(f"   • Sıcaklık: {best_effort['Pred_Temp']:.1f} °C")
        print(f"   • Ayarlar:  PPT {best_effort['Set_PPT_Limit']}W | CO {best_effort['Set_CO_Value']}")

    # --- GÖRSELLEŞTİRME (Scatter Plot) ---
    plt.figure(figsize=(12, 7))
    
    # Renkleri Curve Optimizer değerine göre yapalım (Koyu renk = Daha çok undervolt)
    sns.scatterplot(
        data=df_sim, 
        x='Pred_Temp', 
        y='Pred_Clock', 
        hue='Set_CO_Value', 
        palette='viridis', 
        s=100, 
        alpha=0.8
    )
    
    # Kritik Çizgiler
    plt.axhline(5300, color='red', linestyle='--', linewidth=2, label='5.3 GHz Hedefi')
    plt.axvline(95, color='orange', linestyle='--', label='95°C Sınırı')
    
    plt.title("Yüksek Frekans Arayışı: Hız vs Sıcaklık", fontsize=14)
    plt.xlabel("Tahmini Sıcaklık (°C)", fontsize=12)
    plt.ylabel("Tahmini Hız (MHz)", fontsize=12)
    plt.legend(title='Curve Optimizer')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Çalıştır
simulate_high_frequency_profiles_ordinaryus(model_temp_xgb, model_clock_xgb)

In [ ]:
# --- 1. VERİ YÜKLEME VE HAZIRLIK (FİNAL DÜZELTME) ---
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

def load_and_prep_data_ordinaryus():
    base_path = "/home/egkpnr/code/Ryzen7600X_Analysis/"
    
    print("📂 Veri Dosyaları Yükleniyor...")
    
    files_config = {
        "Fans ve apu down.csv":                 {"ppt": 142, "co": 0,   "boost": 0},
        "95W-115W_80A_140A-20_Per_CCD.csv":     {"ppt": 95,  "co": -20, "boost": 0},
        "-30 CCD PBO 95 +200 mhz.csv":          {"ppt": 95,  "co": -30, "boost": 200},
        "65W-85W_75A_150A-25_Per_CCD.csv":      {"ppt": 65,  "co": -25, "boost": 0},
    }

    df_list = []
    
    for fname, config in files_config.items():
        full_path = os.path.join(base_path, fname)
        if not os.path.exists(full_path): continue
            
        try:
            df = pd.read_csv(full_path, skipinitialspace=True)
            # Sütun Temizliği
            df.columns = df.columns.str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.strip()
            
            # İsim Eşleştirme
            rename_map = {}
            if "Çekirdek Etkili Saatler (avg) MHz" in df.columns: rename_map["Çekirdek Etkili Saatler (avg) MHz"] = "Clock_MHz"
            elif "Core Effective Clocks (avg) MHz" in df.columns: rename_map["Core Effective Clocks (avg) MHz"] = "Clock_MHz"
            elif "Çekirdek Saatler (avg) MHz" in df.columns: rename_map["Çekirdek Saatler (avg) MHz"] = "Clock_MHz"
            
            if "CPU (Tctl/Tdie) °C" in df.columns: rename_map["CPU (Tctl/Tdie) °C"] = "Temp_C"
            
            if "CPU Tüm Güç Tüketimi W" in df.columns: rename_map["CPU Tüm Güç Tüketimi W"] = "Power_W"
            elif "CPU Package Power W" in df.columns: rename_map["CPU Package Power W"] = "Power_W"
            
            if "CPU RPM" in df.columns: rename_map["CPU RPM"] = "Fan_RPM"

            df = df.rename(columns=rename_map)
            
            # Sayısala Çevir
            for col in ['Clock_MHz', 'Temp_C', 'Power_W']:
                if col in df.columns: df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Fan yoksa doldur
            if 'Fan_RPM' not in df.columns: df['Fan_RPM'] = 1500
            
            # Temel Filtreleme (Kritik 3'lüde boş olanı sil)
            df = df.dropna(subset=['Clock_MHz', 'Temp_C', 'Power_W'])
            
            # GHz -> MHz
            if not df.empty and df['Clock_MHz'].max() < 10.0:
                df['Clock_MHz'] = df['Clock_MHz'] * 1000

            # Etiketler
            df['Set_PPT_Limit'] = config['ppt']
            df['Set_CO_Value'] = config['co']
            df['Set_Boost_Override'] = config['boost']
            df['Config_Label'] = fname 
            
            # 3000 MHz altını sil
            df = df[df['Clock_MHz'] > 3000].copy()
            
            if not df.empty: df_list.append(df)
            
        except Exception:
            continue

    if not df_list: return None
    full_df = pd.concat(df_list, ignore_index=True)
    
    # --- C) ORDINARYUS FEATURE ENGINEERING ---
    print(f"🧠 {len(full_df)} satır ham veri işleniyor...")
    
    grouped = full_df.groupby('Config_Label')
    
    # Yeni özellikleri hesapla
    full_df['Temp_Lag1'] = grouped['Temp_C'].shift(1)
    full_df['Temp_Lag2'] = grouped['Temp_C'].shift(2)
    full_df['Clock_Lag1'] = grouped['Clock_MHz'].shift(1)
    full_df['Power_Roll_Mean_5'] = grouped['Power_W'].transform(lambda x: x.rolling(window=5).mean())
    full_df['Aggressiveness_Score'] = full_df['Set_PPT_Limit'] * full_df['Set_CO_Value'].abs()
    
    # --- KRİTİK DÜZELTME BURASI ---
    # Sadece bizim kullandığımız sütunlarda boşluk varsa sil!
    # Diğer 300 tane sütundaki boşluklar umurumuzda değil.
    
    critical_cols = [
        'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 
        'Clock_MHz', 'Temp_C'
    ]
    
    initial_len = len(full_df)
    full_df = full_df.dropna(subset=critical_cols).reset_index(drop=True)
    final_len = len(full_df)
    
    print(f"🎉 TEMİZLİK SONRASI: {final_len} satır veri hazır! (Silinen: {initial_len - final_len})")
    
    return full_df

# Fonksiyonu Çalıştır
df_advanced = load_and_prep_data_ordinaryus()

# --- X ve y DEĞİŞKENLERİNİ HAZIRLA ---
if df_advanced is not None and not df_advanced.empty:
    feature_cols = [
        'Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'Fan_RPM', 
        'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 
        'Aggressiveness_Score'
    ]

    X = df_advanced[feature_cols]
    y_temp = df_advanced['Temp_C']
    y_clock = df_advanced['Clock_MHz']
    
    X_train, X_test, y_temp_train, y_temp_test, y_clock_train, y_clock_test = train_test_split(
        X, y_temp, y_clock, test_size=0.25, random_state=42, shuffle=True
    )
    print("✅ Veri setleri Eğitim (%75) ve Test (%25) olarak ayrıldı.")
else:
    print("❌ HATA: Veri seti boş kaldı.")

In [ ]:
# --- 4. MODEL EĞİTİMİ (ORDINARYUS MOTORU) ---
import xgboost as xgb
import optuna
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

# Optuna'nın kalabalık loglarını kapatalım
optuna.logging.set_verbosity(optuna.logging.WARNING)

def train_xgboost_models_ordinaryus(X_train, y_temp_train, y_clock_train, trials=20):
    print(f"\n🎓 ORDINARYUS MODU: Modeller optimize ediliyor ({trials} deneme)...")
    print("="*60)

    # --- İÇ FONKSİYON: OPTİMİZASYON MANTIĞI ---
    def optimize(X, y, target_name):
        print(f"🔍 {target_name} için en iyi genetik yapı aranıyor...")
        
        def objective(trial):
            # Arama Uzayı (Modelin deneyebileceği sınırlar)
            param = {
                'n_estimators': trial.suggest_int('n_estimators', 500, 1500),
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'max_depth': trial.suggest_int('max_depth', 3, 9),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 6),
                'subsample': trial.suggest_float('subsample', 0.6, 0.9),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.9),
                'gamma': trial.suggest_float('gamma', 0.0, 0.5),
                'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
                'n_jobs': -1,
                'random_state': 42,
                # 'tree_method': 'gpu_hist' # Eğer NVIDIA ekran kartın varsa bu satırı aç
            }
            
            model = xgb.XGBRegressor(**param)
            
            # 5-Fold Cross Validation (Sağlamasını yap)
            cv = KFold(n_splits=5, shuffle=True, random_state=42)
            scores = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error')
            
            return -scores.mean() # Hatayı minimize et

        # Çalışmayı başlat
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=trials, show_progress_bar=True)
        
        print(f"   ✅ {target_name} En İyi RMSE: {study.best_value:.4f}")
        return study.best_params

    # 1. Termal Modeli Optimize Et
    print("   -> Sıcaklık modeli optimize ediliyor...")
    best_params_temp = optimize(X_train, y_temp_train, "Sıcaklık (Temp)")
    
    # 2. Performans Modelini Optimize Et
    print("   -> Frekans modeli optimize ediliyor...")
    best_params_clock = optimize(X_train, y_clock_train, "Hız (Clock)")
    
    print("\n🚀 FİNAL EĞİTİM BAŞLIYOR...")
    
    # En iyi parametrelerle modelleri kur
    model_temp = xgb.XGBRegressor(**best_params_temp)
    model_temp.fit(X_train, y_temp_train)
    
    model_clock = xgb.XGBRegressor(**best_params_clock)
    model_clock.fit(X_train, y_clock_train)
    
    return model_temp, model_clock

# --- FONKSİYONU ÇALIŞTIR ---
# Eğer veriler yüklüyse eğitimi başlat
if 'X_train' in globals() and X_train is not None:
    # Hız için 30 deneme yapalım (Ordinaryus seviyesi için ideal)
    model_temp_xgb, model_clock_xgb = train_xgboost_models_ordinaryus(
        X_train, y_temp_train, y_clock_train, trials=30
    )
    
    print("\n🎉 MODELLER HAZIR VE HAFIZAYA YÜKLENDİ.")
else:
    print("❌ HATA: Önce veri yükleme (load_data...) hücresini çalıştırın!")

In [ ]:
# --- 10. FİNAL RAPORU: PER-CORE OPTİMİZASYON REÇETESİ ---
import pandas as pd

def generate_optimization_recipe(model_c, model_t):
    print("\n" + "="*60)
    print("--- PER-CORE OPTİMİZASYON REÇETESİ (XGBoost Onaylı) ---")
    print("="*60)
    
    # Modelin "En Güvenli ve Hızlı" bulduğu Curve Optimizer değerini arıyoruz
    
    scenarios = []
    for co in [-30, -25, -20, -15, -10, -5]:
        scenarios.append({
            'Set_PPT_Limit': 110,  # Ortalama güçlü bir limit
            'Set_CO_Value': co,
            'Set_Boost_Override': 100,
            'Fan_RPM': 1800,       # DÜZELTİLDİ: Eğitimdeki isimle aynı (Fan_RPM)
            # Ordinaryus Değişkenleri
            'Temp_Lag1': 75, 'Temp_Lag2': 75, 'Clock_Lag1': 5000,
            'Power_Roll_Mean_5': 100,
            'Aggressiveness_Score': 110 * abs(co)
        })
    
    df_recipe = pd.DataFrame(scenarios)
    
    # Sütun sırasını garantiye al (Eğitimdeki 'feature_cols' listesiyle aynı olmalı)
    req_cols = [
        'Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'Fan_RPM', 
        'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 
        'Aggressiveness_Score'
    ]
    
    # Tahmin
    df_recipe['Pred_Clock'] = model_c.predict(df_recipe[req_cols])
    df_recipe['Pred_Temp'] = model_t.predict(df_recipe[req_cols])
    
    # En iyi CO değerini seç (Sıcaklık < 85 ve Hız Maksimum)
    safe_zone = df_recipe[df_recipe['Pred_Temp'] < 85]
    
    if not safe_zone.empty:
        best_row = safe_zone.sort_values(by='Pred_Clock', ascending=False).iloc[0]
        ideal_co = int(best_row['Set_CO_Value'])
        ideal_ppt = int(best_row['Set_PPT_Limit'])
    else:
        ideal_co = -15 # Güvenli liman
        ideal_ppt = 110
        
    print(f"🧠 YAPAY ZEKA ANALİZİ:")
    print(f"   İşlemcinizin silikon kalitesine göre ideal Global CO: {ideal_co}")
    print("-" * 60)
    
    print("💊 REÇETE (BIOS AYARLARI):")
    
    print(f"   1. [Global C-State Control]: Enabled")
    print(f"   2. [CPPC]: Enabled")
    print(f"   3. [CPPC Preferred Cores]: Enabled")
    print(f"   4. [PPT Limit]: {ideal_ppt} W (Tatlı Nokta)")
    print(f"   5. [CPU Boost Override]: +100 MHz")
    print(f"   6. [Curve Optimizer]: PER CORE")
    
    print("\n   🎯 ÇEKİRDEK BAZLI DEĞERLER:")
    print(f"      • En İyi İki Çekirdek (Genelde Core 0, 1): {ideal_co + 5}  (Örn: -30 ise -25 yap)")
    print(f"      • Diğer Tüm Çekirdekler (Core 2, 3, 4, 5): {ideal_co}     (Modelin Önerisi)")
    
    print("\n⚠️ NOT: 'En iyi çekirdekler' Windows Görev Yöneticisi veya Ryzen Master'da")
    print("        yıldız (*) veya altın nokta ile işaretli olanlardır.")
    print("="*60)

# Reçeteyi Yazdır
generate_optimization_recipe(model_clock_xgb, model_temp_xgb)

In [ ]:
# --- 10. FİNAL RAPORU: PER-CORE OPTİMİZASYON REÇETESİ (FİNAL HİBRİT) ---
import pandas as pd

def generate_optimization_recipe_hybrid(model_c, model_t):
    print("\n" + "="*60)
    print("--- PER-CORE OPTİMİZASYON REÇETESİ (Hibrit Güvenlik) ---")
    print("="*60)
    
    # Modele soruyoruz: "Genel olarak en iyi CO değeri nedir?"
    scenarios = []
    for co in [-30, -25, -20, -15]:
        scenarios.append({
            'Set_PPT_Limit': 110,
            'Set_CO_Value': co,
            'Set_Boost_Override': 100,
            'Fan_RPM': 1800,
            'Temp_Lag1': 75, 'Temp_Lag2': 75, 'Clock_Lag1': 5000,
            'Power_Roll_Mean_5': 100,
            'Aggressiveness_Score': 110 * abs(co)
        })
    
    df_recipe = pd.DataFrame(scenarios)
    req_cols = ['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'Fan_RPM', 
                'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 'Aggressiveness_Score']
    
    df_recipe['Pred_Clock'] = model_c.predict(df_recipe[req_cols])
    
    # Modelin en yüksek hız için önerisi (Muhtemelen -30 çıkacak)
    best_row = df_recipe.sort_values(by='Pred_Clock', ascending=False).iloc[0]
    ai_suggestion = int(best_row['Set_CO_Value'])
    
    print(f"🧠 YAPAY ZEKA ÖNERİSİ (Global): {ai_suggestion}")
    print(f"🛡️  DONANIM GERÇEĞİ: Core 5 bilinen zayıf halkadır.")
    print("-" * 60)
    
    print("💊 NİHAİ BIOS REÇETESİ:")
    print(f"   1. [PPT Limit]: {int(best_row['Set_PPT_Limit'])} W")
    print(f"   2. [CPU Boost Override]: +100 MHz")
    print(f"   3. [Curve Optimizer]: Per Core (Çekirdek Bazlı)")
    
    print("\n   🎯 ÇEKİRDEK DAĞILIMI:")
    
    # Zeka ve Güvenliği Birleştiriyoruz
    print(f"      • Core 0 & 1 (Yıldızlılar):  {ai_suggestion + 5}   (Örn: -25 -> Yüksek frekans stabilitesi için)")
    print(f"      • Core 2, 3, 4 (Sağlamlar):  {ai_suggestion}      (Örn: -30 -> Maksimum verim)")
    print(f"      • Core 5 (Zayıf Halka):      -20      (Manuel Güvenlik Kilidi)")
    
    print("\n✅ YORUM: Bu ayar, sağlam çekirdeklerin potansiyelini (Model verisi)")
    print("          kullanırken, Core 5'in çökmesini (Tecrübe verisi) engeller.")
    print("="*60)

# Çalıştır
generate_optimization_recipe_hybrid(model_clock_xgb, model_temp_xgb)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

# Test edilecek ağaç sayıları
tree_counts = [10, 50, 100, 200, 500]
results = []

print("Optimum ağaç sayısı testi yapılıyor...")

for n in tree_counts:
    # Modeli farklı ağaç sayılarıyla eğit
    rf_test = RandomForestRegressor(n_estimators=n, random_state=42, n_jobs=-1)
    rf_test.fit(X_train, y_temp_train) # Sıcaklık tahmini için test
    
    # Test seti üzerindeki hatayı ölç
    preds = rf_test.predict(X_test)
    mae = mean_absolute_error(y_temp_test, preds)
    results.append(mae)
    print(f"Ağaç Sayısı: {n} -> Hata (MAE): {mae:.4f} Derece")

# Sonucu Görselleştir
plt.figure(figsize=(10, 6))
plt.plot(tree_counts, results, marker='o', linestyle='-', color='b')
plt.title('Ağaç Sayısı vs Hata Oranı (Dirsek Testi)')
plt.xlabel('Ağaç Sayısı (n_estimators)')
plt.ylabel('Ortalama Hata (Daha düşük daha iyi)')
plt.grid(True)
plt.show()